In [22]:
import tensorflow as tf 
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import matplotlib.pyplot as plt
import os 

In [2]:
# check tf version
tf.__version__

'2.15.0'

In [19]:
# check the size of train, test and validation

def check_size_data():
    base_dir = "./data/"
    for i in ['train', 'valid', 'test']:
        print(f"for {i}") 
        for file in os.listdir(os.path.join(base_dir, i)): 
            print(file)
            print(f'{file} file has {len(os.listdir(os.path.join(base_dir + i, file )))}')
        print("=====================================") 
        # print(f'{i} has {len(os.path.join(base_dir, i))} photos')

check_size_data()

for train
Ayam Betutu
Ayam Betutu file has 42
Beberuk Terong
Beberuk Terong file has 42
Coto Makassar
Coto Makassar file has 42
Gudeg
Gudeg file has 42
Kerak Telor
Kerak Telor file has 42
Mie Aceh
Mie Aceh file has 42
Nasi Kuning
Nasi Kuning file has 42
Nasi Pecel
Nasi Pecel file has 42
Papeda
Papeda file has 42
Pempek
Pempek file has 42
Peuyeum
Peuyeum file has 42
Rawon
Rawon file has 42
Rendang
Rendang file has 42
Sate Madura
Sate Madura file has 42
Serabi
Serabi file has 42
Soto Banjar
Soto Banjar file has 42
Soto Lamongan
Soto Lamongan file has 42
Tahu Sumedang
Tahu Sumedang file has 42
for valid
Ayam Betutu
Ayam Betutu file has 4
Beberuk Terong
Beberuk Terong file has 4
Coto Makassar
Coto Makassar file has 4
Gudeg
Gudeg file has 4
Kerak Telor
Kerak Telor file has 4
Mie Aceh
Mie Aceh file has 4
Nasi Kuning
Nasi Kuning file has 4
Nasi Pecel
Nasi Pecel file has 4
Papeda
Papeda file has 4
Pempek
Pempek file has 4
Peuyeum
Peuyeum file has 4
Rawon
Rawon file has 4
Rendang
Rendang file h

In [24]:
len(os.listdir("./data/train/"))

18

In [27]:
def image_generator(training_dir, validation_dir):
    train_datagen = tf.keras.utils.image_dataset_from_directory(
         rescale= 1./255.,
         rotation_range=40,
         width_shift_range=0.2,
         height_shift_range=0.2,
         shear_range=0.2,
         zoom_range=0.2,
         horizontal_flip=True,
         fill_mode='nearest'
    )
    # to directory
    train_generator = train_datagen.flow_from_directory(
        directory = training_dir,
                     batch_size=20,
                     class_mode='categorical',
                     target_size=(150, 150)
    )

    # for validation
    validation_datagen = tf.keras.utils.image_dataset_from_directory(
         rescale= 1./255.,
         rotation_range=40,
         width_shift_range=0.2,
         height_shift_range=0.2,
         shear_range=0.2,
         zoom_range=0.2,
         horizontal_flip=True,
         fill_mode='categorical'
    )
    # to directory
    validation_generator = validation_datagen.flow_from_directory(
        directory = validation_dir,
        batch_size=20,
        class_mode='binary',
        target_size=(150, 150)
    )
    return train_generator, validation_generator

In [50]:
training_dir = "./data/train/"
validatiion_dir = "./data/valid/" 
test_data = "./data/test/"

train_gen, valid_gen = image_generator(training_dir, validatiion_dir)
# creating the model 
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(150, 150, 3)), 
    tf.keras.layers.Conv2D(filters = 128, kernel_size = (3, 3), strides = (1, 1), activation = 'relu', padding="same"),
    tf.keras.layers.Conv2D(filters = 64, kernel_size = (3, 3), strides = (1, 1), activation = 'relu', padding="same"),
    tf.keras.layers.Conv2D(filters = 32, kernel_size = (3, 3), strides = (1, 1), activation = 'relu', padding="same"),
    tf.keras.layers.Conv2D(filters = 8, kernel_size = (3, 3), strides = (1, 1), activation = 'relu', padding="same"),
    # flltern
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 512, activation='relu'), 
    tf.keras.layers.Dense(units = 18, activation='softmax'), 
])

Found 756 images belonging to 18 classes.
Found 72 images belonging to 18 classes.


In [58]:
# 

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Changed loss function
model.summary()
# model.fit(train_gen, epochs=20, validation_data=valid_gen)


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 148, 148, 128)     3584      
                                                                 
 conv2d_33 (Conv2D)          (None, 146, 146, 64)      73792     
                                                                 
 conv2d_34 (Conv2D)          (None, 144, 144, 32)      18464     
                                                                 
 conv2d_35 (Conv2D)          (None, 142, 142, 8)       2312      
                                                                 
 flatten_8 (Flatten)         (None, 161312)            0         
                                                                 
 dense_16 (Dense)            (None, 512)               82592256  
                                                                 
 dense_17 (Dense)            (None, 18)               

In [59]:
# using transfer learning 
IMG_SHAPE = (150, 150) + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


9406464/9406464 [==============================] - 21s 2us/step


In [64]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 75, 75, 32)           864       ['input_11[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 75, 75, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 75, 75, 32)           0         ['bn_Conv1[

In [60]:
base_model.trainable = False

In [67]:
# prediction_layer = tf.keras.layers.Dense(18, activation='sigmoid')
# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# inputs = tf.keras.layers.Input(shape=(150, 150, 3))
# # x = data_augmentation(inputs)
# # x = preprocess_input(x)
# # x = base_model(x, training=False)
# # x = global_average_layer(x)
# # x = tf.keras.layers.Dropout(0.2)(x)
# # outputs = prediction_layer(x)
# model2 = tf.keras.Model(inputs, prediction_layer)
# model2.summary()



# Definisikan layer-layer yang diperlukan
prediction_layer = tf.keras.layers.Dense(18, activation='sigmoid')
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# Definisikan input layer
inputs = tf.keras.layers.Input(shape=(150, 150, 3))
x = base_model(inputs, training = False)
# Gunakan layer-layer yang telah didefinisikan sebelumnya untuk memproses input
x = global_average_layer(x)
outputs = prediction_layer(x)

# Buat model dengan input dan output layer yang telah didefinisikan
model2 = tf.keras.Model(inputs, outputs)
model2.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_4  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_22 (Dense)            (None, 18)                23058     
                                                                 
Total params: 2281042 (8.70 MB)
Trainable params: 23058 (90.07 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [68]:
tf.keras.utils.plot_model(model2, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [72]:
base_learning_rate = 0.0001
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),loss= 'categorical_crossentropy',metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')])
history2 = model2.fit(train_gen,epochs=20, validation_data=valid_gen)

Epoch 1/20
38/38 [==============================] - 11s 252ms/step - loss: 1.2722 - accuracy: 0.9308 - val_loss: 1.5194 - val_accuracy: 0.9190
Epoch 2/20
38/38 [==============================] - 5s 136ms/step - loss: 1.2380 - accuracy: 0.9311 - val_loss: 1.4552 - val_accuracy: 0.9136
Epoch 3/20
38/38 [==============================] - 5s 130ms/step - loss: 1.2478 - accuracy: 0.9308 - val_loss: 1.3635 - val_accuracy: 0.9306
Epoch 4/20
38/38 [==============================] - 5s 129ms/step - loss: 1.2156 - accuracy: 0.9353 - val_loss: 1.4109 - val_accuracy: 0.9290
Epoch 5/20
38/38 [==============================] - 5s 129ms/step - loss: 1.1661 - accuracy: 0.9375 - val_loss: 1.4242 - val_accuracy: 0.9252
Epoch 6/20
38/38 [==============================] - 5s 137ms/step - loss: 1.1471 - accuracy: 0.9376 - val_loss: 1.3038 - val_accuracy: 0.9275
Epoch 7/20
38/38 [==============================] - 5s 140ms/step - loss: 1.0852 - accuracy: 0.9392 - val_loss: 1.2233 - val_accuracy: 0.9344
Epoch

In [ ]:
# note : not finished yet

In [ ]:
# use vgg-16

In [ ]:
# plot accuracy, validation losss and